In [4]:
import pandas as pd
import numpy as np
from ortools.sat.python import cp_model
import requests

In [10]:
api_key = "keyB5I5hLN8iKV5y3"
base_id = 'app4kNRRTwN0JBBNk'
table_name="Schedule"
url = "https://api.airtable.com/v0/" + base_id + "/" + table_name
params={"view": 'Schedule CSV Export'}

In [11]:
response = requests.get(url, params=params, headers=headers)
airtable_response = response.json()

In [12]:
airtable_response

{'records': [{'id': 'rec2FZjGPACQ7sOrW',
   'fields': {'Volunteer Shift': 'TUE 9-11AM',
    'All Available Pantry Volunteers': ['rec7w5uvbwKhobPR6',
     'recIuKkSW3qRYmKln',
     'recR8axz2SjZmhXJR',
     'recUKPtV5IBbQdKTX',
     'recXiy72eQ6WzWahD',
     'recJFAknC70oaJvli',
     'recXiCUcvFV3RyQna',
     'recGynaCIvr3lvx41',
     'rec1ZmCrwguPS3WrS'],
    'All Available Recovery Volunteers': ['recR8axz2SjZmhXJR'],
    'Pantry Shift Notes': 'Restock\n(Ed is out on 6/22 // Baldemar starting in July)\n',
    'Assigned Pantry Volunteers': ['recJFAknC70oaJvli',
     'recIuKkSW3qRYmKln',
     'recXrgbkevVYGWfsD',
     'recGynaCIvr3lvx41',
     'rec1ZmCrwguPS3WrS'],
    'Staff': ['recCLTprcK6toA17i', 'recVh5Zsjin7faBZe'],
    'Shift length (hours)': 2,
    'Shift Start Date': '2021-06-01',
    'Check In': ['recSde6QkExGd3ag7',
     'rece101Nw51YZJAhu',
     'recizZVslrEEpbNIT',
     'recRo7GWOGVFw56Jd',
     'recDljIXvGGWUKmhL',
     'recUrNWFRDANZodgO'],
    'Last Modified': '2021-06-29T

In [15]:
airtable_records = airtable_response['records']
params = ()
airtable_records = []
run = True
while run is True:
  response = requests.get(url, params=params, headers=headers)
  airtable_response = response.json()
  airtable_records += (airtable_response['records'])
  if 'offset' in airtable_response:
     run = True
     params = (('offset', airtable_response['offset']),)
  else:
     run = False

In [19]:
airtable_rows = [] 
airtable_index = []
for record in airtable_records:
    airtable_rows.append(record['fields'])
    airtable_index.append(record['id'])
shift_records = pd.DataFrame(airtable_rows, index=airtable_index)
shift_records['Volunteer Shift']

rec0UugasEZTSNtBQ        TUE 2-4PM
rec2FZjGPACQ7sOrW       TUE 9-11AM
recB9R9N9IuE5VW07       WED 9-11AM
recFNCey5xeNOgVFO        TUE 6-8PM
recFcl4l5kYJLxSr2      TUE 10-12PM
recFziQpuyQZu5to6        THU 6-8PM
recHBSuOru0gXP3Nm        TUE 5-7PM
recJoKnZcvmfXdpZV        WED 1-3PM
recNbZnMFc3aSnS1a        TUE 3-5PM
recO47ebZhK7ln8Qp        TUE 1-3PM
recPwTKeZxDiwBF91       SAT 11-1PM
recZveGeDndZoz52g       WED 11-1PM
recZwedHESGSegMj6    SAT 10AM-12PM
recaMCe6dGnjN7dqJ       TUE 12-2PM
recaqAOsCAlYJ5rGD       WED 8-10AM
recbaaw3WyzJQnbxf    FRI 10AM-12PM
recdOrWWkqnfqvvr6         MON 9-11
recf0Ojfa6ju6DB7Y      WED 10-12PM
rechVMljbsqhnBXK2        THU 5-7PM
reck7nbgZK8UIQinB     WED 6:30-8AM
recsGQje46Fyte3FJ        THU 4-6PM
recvAoAR58Y4tz07t       TUE 11-1PM
recxEKqMG16kK8weX        TUE 4-6PM
reczreGEbMkM0Cbk1       SAT 12-2PM
Name: Volunteer Shift, dtype: object

In [14]:
# data imports
def read_csvs(response_csv, shift_csv, lower_bounds):
    #responses = pd.read_csv("autoscheduler_Export.csv")
    #shifts = pd.read_csv("shifts.csv")
    #recovery_shifts = pd.read_csv('recoveryshifts.csv')
    responses = pd.read_csv(response_csv)
    shifts = pd.read_csv(shift_csv)
    hours = responses[['Full Name', 'Volunteer Weekly Commitment']].set_index('Full Name')
    assignments = run_csp(responses, shifts, hours, lower_bounds)
    return assignments

In [15]:
#data transformation
def Transform_Responses(shifts, response_df):
    all_shifts = list(shifts['Volunteer Shift'])
    columns = np.append(['Full Name'], all_shifts)
    availability = pd.DataFrame(columns=columns)
    for i in np.arange(response_df.shape[0]):
        mapping = []
        for j in all_shifts:
            if j in response_df.iloc[i,2]:
                mapping.append(1)
            else:
                mapping.append(0)
            row_data = np.append(response_df.iloc[i,0], mapping)
        availability.loc[len(availability)] = row_data
    return availability

In [58]:
def run_csp(responses, shifts, hours, lower_bounds):
    availability = Transform_Responses(shifts, responses).set_index('Full Name')

    #define decision variables
    all_shifts = list(shifts['Volunteer Shift'])
    all_volunteers = list(responses['Full Name'])
    variables = []
    variable_domain = [0, 1]
    
    #Shift times Ex: MON 10-12PM
    for s in all_shifts:
        for v in all_volunteers:
            if int(availability.loc[v,s]) == 1:
                variables.append(v + ';' + s)
    print(all_shifts[0])            
    shift_Sum = []
    for i in all_shifts:
        one_shift = []
        for j in variables:
            if j.split(';')[1] in i:
                one_shift.append(j)
        shift_Sum.append(one_shift)
        
    vol_Sum = []
    for i in all_volunteers:
        one_user = []
        for j in variables:
            if j.split(';')[0] in i:
                one_user.append(j)
        vol_Sum.append(one_user)
        
    # CSP model
    model = cp_model.CpModel()

    gvars = list()
    for var in variables:
        var = model.NewIntVar(0, 1, var)
        gvars.append(var)

    vol_Sum = []
    for i in all_volunteers:
        one_user = []
        for j in gvars:
            if j.Name().split(';')[0] in i:
                one_user.append(j)
        vol_Sum.append(one_user)
        
    shift_Sum = []
    for i in all_shifts:
        one_shift = []
        for j in gvars:
            if j.Name().split(';')[1] in i:
                one_shift.append(j)
        shift_Sum.append(one_shift)
        
    #constraints based on volunteer hours (volunteer shifts)
    #lower bound of 1 on each volunteer (to ensure every volunteer has one shift):
    for vs in vol_Sum:
        model.Add(sum(vs) >= 1)
        
  
    #upper bound constraint for each user (volunteer weekly commitment)
    for vs in vol_Sum:
        vol = vs[0].Name().split(';')[0]
        k = hours.loc[vol][0]
        model.Add(sum(vs) <= k)

    print('volunteer constraints created!')

    #shift based constraints (filling each shift but not overfilling)
    #lower bound (of 1) on shifts
    print(len(shift_Sum))
    for shift in range(len(shift_Sum)):
        if type(lower_bounds) == int:
            model.Add(sum(shift_Sum[shift]) >= lower_bounds)
            model.Add(sum(shift_Sum[shift]) <= 5)
        else:
            model.Add(sum(shift_Sum[shift]) >= lower_bounds[shift])
            model.Add(sum(shift_Sum[shift]) <= 5)
        
        
    print('shift constraints created!')

    #Maximize number of volunteers working
    #Adding a term that maximizes median of each shift
    model.Maximize(sum(gvars))
        
    #Get solution
    #solutions = problem.getSolution()
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    #pull solutions into dataframe
    assigned = []
    for var in gvars:
        if solver.Value(var) == 1:
            assigned.append(var.Name())

    assigned_dict = {}
    for i in all_volunteers:
        assigned_dict[i] = []
        for j in assigned:
            splits = j.split(';')
            if splits[0] == i:
                assigned_dict[i].append(splits[1])
                
    for i in assigned_dict:
        assigned_dict[i] = ', '.join(assigned_dict[i])

    assignments = pd.DataFrame.from_dict(assigned_dict, orient='index',
                           columns=['Assigned Pantry Shifts'])

    assignments.reset_index(inplace=True)
    assignments.rename(columns={'index': 'Full Name'}, inplace=True)

    #Read file to csv.

    #assignments.to_csv('assignments.csv')
    return assignments

In [59]:
assignments = "autoscheduler_Export.csv"
shifts = "shifts.csv"
lower_bounds = []
for i in range(34):
    lower_bounds.append(3)
assigned = read_csvs(assignments, shifts, 1)
assigned

MON 10-12PM
volunteer constraints created!
34
shift constraints created!


,Full Name,Assigned Pantry Shifts
0,Gillian Edgelow,"FRI 10-12AM, SAT 10-12AM"
1,Giulia de Azeredo Valdejao,THU 1-3PM
2,Sierra Manske,THU 10-12AM
3,Ivet Ramirez,"TUE 10-12PM, THU 4-6PM"
4,Alison White,WED 8-10AM
...,...,...
102,Joaquin Jamieson,WED 10-12AM
103,Kiran Bengard,WED 7-9AM
104,Amy Cranch,FRI 10-12AM
105,Leah Varghese,THU 6-8PM


In [50]:
sorted_assigned = assigned.groupby("Assigned Pantry Shifts").agg(len).sort_values("Full Name", ascending=False)
sorted_assigned

,Full Name
Assigned Pantry Shifts,
WED 1-3PM,4
THU 10-12AM,4
WED 10-12AM,4
WED 9-11AM,3
FRI 10-12AM,3
...,...
"FRI 1-3PM, SAT 1-3PM",1
"TUE 1-3PM, WED 8-10AM",1
TUE 10-12PM,1


In [51]:
sorted_assigned.value_counts()

Full Name
1            52
3             9
2             8
4             3
dtype: int64

In [33]:
pd.read_csv("autoscheduler_Export.csv")

,Full Name,Volunteer Weekly Commitment,All Pantry Shift Availabilities,Recovery Shift Availabilities
0,Gillian Edgelow,2,"FRI 10-12AM,FRI 11-1PM,FRI 12-2PM,SAT 10-12AM,...",NaN
1,Giulia de Azeredo Valdejao,1,"THU 1-3PM,THU 2-4PM,MON 10-12PM",NaN
2,Sierra Manske,1,"WED 10-12AM,WED 11-1PM,THU 10-12AM,FRI 11-1PM",NaN
3,Ivet Ramirez,2,"THU 4-6PM,THU 5-7PM,MON 10-12PM,TUE 10-12PM",NaN
4,Alison White,1,"WED 8-10AM,TUE 10-12PM,WED 1-3PM",NaN
...,...,...,...,...
102,Joaquin Jamieson,1,"TUE 12-2PM,TUE 1-3PM,FRI 11-1PM,SAT 10-12AM,WE...",NaN
103,Kiran Bengard,1,"WED 11-1PM,WED 12-2PM,SAT 11-1PM,WED 7-9AM,FRI...",NaN
104,Amy Cranch,1,"FRI 10-12AM,TUE 9-11AM,TUE 2-4PM,THU 12-2PM,TH...",NaN
105,Leah Varghese,1,"THU 6-8PM,WED 1-3PM,THU 11-1PM,FRI 12-2PM,TUE ...",NaN


In [34]:
a = pd.DataFrame(pd.read_csv("shifts.csv")['Volunteer Shift'])
a.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Volunteer Shift</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>MON 10-12PM</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>TUE 9-11AM</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>TUE 10-12PM</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>TUE 11-1PM</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>TUE 12-2PM</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>TUE 1-3PM</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>TUE 2-4PM</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>TUE 3-5PM</td>\n    </tr>\n    <tr>\n      <th>8</th>\n      <td>TUE 4-6PM</td>\n    </tr>\n    <tr>\n      <th>9</th>\n      <td>TUE 5-7PM</td>\n    </tr>\n    <tr>\n      <th>10</th>\n      <td>WED 7-9AM</td>\n    </tr>\n    <tr>\n      <th>11</th>\n      <td>WED 8-10AM</td>\n    </tr>\n    <tr>\n      <th>12</th>\n      <td>W

In [37]:
assigned.groupby("Assigned Pantry Shifts").agg(len)

,Full Name
Assigned Pantry Shifts,
FRI 1-3PM,1
"FRI 1-3PM, SAT 12-2PM",1
FRI 10-12AM,3
"FRI 10-12AM, SAT 10-12AM",1
FRI 11-1PM,1
...,...
"WED 7-9AM, THU 5-7PM, SAT 11-1PM",1
"WED 7-9AM, WED 8-10AM, SAT 12-2PM",1
WED 8-10AM,2
